<a href="https://colab.research.google.com/github/skojaku/adv-net-sci/blob/main/node2vec_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/skojaku/fastnode2vec.git
# Using pip (with plotting support)
!pip install igraph cairocffi pycairo python-igraph

In [ ]:
import gensim
import igraph as ig
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Load the karate club network
g = ig.Graph.Famous("Zachary")
A = g.get_adjacency_sparse()

# Get community labels (Mr. Hi = 0, Officer = 1)
labels = np.array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
g.vs["label"] = labels

# Visualize the network
palette = sns.color_palette().as_hex()
ig.plot(g, vertex_color=[palette[label] for label in labels], bbox=(300, 300))

In [ ]:
import fastnode2vec

model = fastnode2vec.Node2Vec(
    p = 1.0,
    q = 1.0,
    walk_length = 100,
    window_length= 5,
)
model.fit(A)
emb = model.transform(dim=64)


In [ ]:
emb

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
xy = pca.fit_transform(emb)

sns.set_style("white")
fig, ax = plt.subplots(figsize=(5,5))
ax = sns.scatterplot(x=xy[:,0], y=xy[:,1], s = 100, hue = labels)
ax.set_xticks([])
ax.set_yticks([])
sns.despine()

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3, random_state=0).fit(emb)
detected_labels = kmeans.labels_


# Visualize the network
palette = sns.color_palette().as_hex()
ig.plot(g, vertex_color=[palette[label] for label in detected_labels], vertex_label = list(labels), bbox=(300, 300))

In [ ]:
import networkx as nx
h = ig.Graph.from_networkx(nx.les_miserables_graph())
A_lesmis = h.get_adjacency_sparse()

model = fastnode2vec.Node2Vec(
    p = 1.0,
    q = 0.1,
    walk_length = 300,
    window_length= 10,
)
model.fit(A_lesmis)
emb = model.transform(dim=16)

from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=4, random_state=0).fit(emb)
detected_labels = kmeans.labels_


# Visualize the network
palette = sns.color_palette().as_hex()
ig.plot(h, vertex_color=[palette[label] for label in detected_labels], bbox=(500, 500))